In [ ]:
base_path = 'dialog_data/dailydialog/'

def read_data(data_path):
    src = []
    trg = []
    with open(base_path+data_path) as f:
        train_data = f.readlines() #f.read().splitlines()
        #print(len(train_data))
        for line in train_data:
            utterences = line.split('__eou__')
            del utterences[-1] #last item is '\n'
            src_utterence = []
            for idx in range(len(utterences)-1):
                # src_utterence=src_utterence+'</s>'+trg_utterence
                src_utterence.append(utterences[idx])
                
                if(len(src_utterence)>5):
                   src_utterence = src_utterence[-5:]
                
                src.append('</s>'.join(src_utterence))
                trg_utterence = utterences[idx+1].strip()
                #trg_utterence = trg_polite_dict[utterences[idx+1]]
                trg.append(trg_utterence)
                
        return src,trg
train_src, train_trg = read_data('train/dialogues_train.txt')
print(len(train_src), len(train_trg))
dev_src, dev_trg = read_data('validation/dialogues_validation.txt')
print(len(dev_src), len(dev_trg))
test_src, test_trg = read_data('test/dialogues_test.txt')
print(len(test_src), len(test_trg))

In [ ]:
import pickle

In [ ]:
vanilla = list()
with open('dialog/daily_dialog/without-finetune/dlg_responses_wo_pred_lists.pkl', 'rb') as f:
    vanilla = pickle.load(f)
print(len(vanilla))

finetune = list()
with open('dialog/daily_dialog/finetune/dlg_responses_direct_pred_lists.pkl', 'rb') as f:
    finetune = pickle.load(f)
print(len(finetune))

finetune_taggen = list()
with open('dialog/daily_dialog/finetune-polite/Blenderbot_polite_pred_tag-gen.pkl', 'rb') as f:
    finetune_taggen = pickle.load(f)
print(len(finetune_taggen))

finetune_ours = list()
with open('dialog/daily_dialog/finetune-polite/polite_dlg_responses_direct_pred_lists.pkl', 'rb') as f:
    finetune_ours = pickle.load(f)
print(len(finetune_ours))

In [ ]:
for idx in range(97,99):
    print('Context')
    print(10*'-')
    print(test_src[idx])
    
    print('\nUtterance')
    print(10*'-')
    print(test_trg[idx])
    
    print('\nVanila')
    print(10*'-')
    print(vanilla[idx])
    
    print('\nFinetune')
    print(10*'-')
    print(finetune[idx])
    
    print('\nFinetune Tag-gen')
    print(10*'-')
    print(finetune_taggen[idx].replace('averaging',''))
    
    print('\nFinetune Ours')
    print(10*'-')
    print(finetune_ours[idx])
    
    print('\n')
    print(idx)
    
    print(50*'=')
    print('\n')

In [ ]:
import random
custom_list = random.sample(range(0, 1000), 100)
print(custom_list)

In [ ]:
import pandas as pd

In [ ]:
eval_data = list()
for idx in custom_list:
    dict1 = {'id': 'vanilla', 'context': test_src[idx], 'utterance': test_trg[idx], 'pred_utterance': vanilla[idx]}
    dict2 = {'id': 'finetune', 'context': test_src[idx], 'utterance': test_trg[idx], 'pred_utterance': finetune[idx]}
    dict3 = {'id': 'finetune_taggen', 'context': test_src[idx], 'utterance': test_trg[idx], 'pred_utterance': finetune_taggen[idx]}
    dict4 = {'id': 'finetune_ours', 'context': test_src[idx], 'utterance': test_trg[idx], 'pred_utterance': finetune_ours[idx]}
    dict_list = [dict1,dict2,dict3,dict4] 
    random.shuffle(dict_list)
    for d in dict_list:
        eval_data.append(d)
df = pd.DataFrame(eval_data)
df.to_csv('human_evals.csv')

In [ ]:
import pandas as pd

SHEET_ID = '1sS48z1jTin_eyKbHF5oCwY2-xA-cgeEd6cLlhPlTJUA'
SHEET_NAME = 'Polite_Chatbot_Human_Evals'
url = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'
df = pd.read_csv(url)
print(df.head())

In [ ]:
print(df['id'].values.tolist()[199], df['politeness'].values.tolist()[199], df['coherent-to-context'].values.tolist()[199], df['grammatical-quality'].values.tolist()[199])

In [ ]:
def avg_score(id_list, metric_list):
    vanila = list()
    finetune = list()
    finetune_taggen = list()
    finetune_ours = list()
    for idx, i in enumerate(id_list):
        if i == 'vanilla':
            vanila.append(int(metric_list[idx]))
        elif i == 'finetune':
            finetune.append(int(metric_list[idx]))
        elif i == 'finetune_taggen':
            finetune_taggen.append(int(metric_list[idx]))
        elif i == 'finetune_ours':
            finetune_ours.append(int(metric_list[idx]))
    return sum(vanila)/len(vanila),sum(finetune)/len(finetune),sum(finetune_taggen)/len(finetune_taggen),sum(finetune_ours)/len(finetune_ours) 

In [ ]:
res = avg_score(df['id'].values.tolist()[0:200], df['politeness'].values.tolist()[0:200])
print(res)

In [ ]:
res = avg_score(df['id'].values.tolist()[0:200], df['coherent-to-context'].values.tolist()[0:200])
print(res)

In [ ]:
res = avg_score(df['id'].values.tolist()[0:200], df['grammatical-quality'].values.tolist()[0:200])
print(res)